In [141]:
import re

In [142]:
samples = [
    'FAx(P(x) -> ~M(x))',
    'FAxFAyFAz(M(x) -> ~ M(y) & ~M(z))',
    '~(P(x) | Q(x))',
    '~FAx(P(x))',
    '~~P(x)'
]

## Step 1: Elimiate_implication

In [143]:
expression = '(P | Q) -> (R)'
quantifiers_filter = re.compile('([A-z][A-Z].)')
def eliminate_implication(expression):
    quants = re.findall(quantifiers_filter, expression)
    sentence = re.sub(r'([A-z][A-Z].)', '', expression)
    parts = re.split('->', sentence)
    if len(quants) == 0:
        new_sentence = '~' + parts[0] +'|'+ parts[1]
    else:
        new_sentence = parts[0][0] + '~'+ parts[0][1:] +'|'+ parts[1]
    return ''.join(quants) + new_sentence

In [144]:
eliminate_implication(samples[1])

'FAxFAyFAz(~M(x) | ~ M(y) & ~M(z))'

## 2: Move negation inside

In [185]:
expression =  '~(P(x) | Q(x))'
def move_negation_inside(expression):
    expression = expression.replace('~~', '')
    quants = re.findall(quantifiers_filter, expression)
    expression = re.sub(r'([A-z][A-Z].)', '', expression)
    print(expression)

    while '~(' in expression:
        start_idx = expression.index('~(')
        end_idx = start_idx + 1
        bracket_counter = 0
        for i in range(start_idx + 1, len(expression)):
            if expression[i] == '(':
                bracket_counter += 1
            elif expression[i] == ')':
                bracket_counter -= 1
                if bracket_counter == 0:
                    end_idx = i
                    break

        sub_expr = expression[start_idx + 2:end_idx]
        opreators_list = re.findall('\ \| |\ & ', sub_expr)
        if len(opreators_list) == 0:
            new_sentence = '~' + expression
        elif opreators_list[0] == ' | ':
            opposite_opreator_list = [' & ']
            parts = re.split('\ \| |\ & ', sub_expr)
            for i in range(len(parts)):
                if parts[i][0] == '~' or parts[i][1] == '~':
                    parts[i].replace('~', '')
                else:
                    parts[i] = '~'+ parts[i]
            new_sentence = parts[0] +opposite_opreator_list[0]+ parts[1]

        elif opreators_list[0] == ' & ':
            opposite_opreator_list = [' | ']
            parts = re.split('\ \| |\ & ', sub_expr)
            for i in range(len(parts)):
                if parts[i][0] == '~' or parts[i][1] == '~':
                    parts[i].replace('~', '')
                else:
                    parts[i] = '~'+ parts[i]
            new_sentence = parts[0] +opposite_opreator_list[0]+ parts[1]
        expression = expression[:start_idx] + new_sentence + expression[end_idx+1:]
    
    quants_map = {'FA': 'TE', 'TE': 'FA'}
    opposite_quants_list = [quants_map[op] if op in quants_map else op for op in quants]

    expression = expression.replace('~FA', 'TE')
    expression = expression.replace('~TE', 'FA')
    return ''.join(opposite_quants_list) + expression

In [ ]:
exp = '~(P(x))'
move_negation_inside(exp)